## Implement a hash table and upload your code to github:

- Use the multiplication AND division method for your hash function
- Note your code should be generic enough to allow for ANY hash function
- For simplicity assume your keys are integers and the values (data) are integers
- Use collision resolution by chaining
- Use a doubly linked list and you must write your own (so for example you can't use "list" in C++)
- You are only allowed to use C-style array's for this implementation (so for example no C++ vectors)
- Your Hash table should grow and shrink
- When it's full double the array size and re-hash everything
- When it's becoming empty e.g. 1/4 empty, then half the size of the array and re-hash everything

In [3]:
class Node:
    # Represents a node in the doubly linked list used for chaining.
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None


class HashTable:
    def __init__(self, capacity=8, hash_function=None):
        # Initialize the hash table with a given capacity and optional custom hash function.
        self.capacity = capacity
        self.size = 0
        self.table = [None] * capacity
        self.A = 0.6180339887  # Multiplication method constant
        self.hash_function = hash_function if hash_function else self._default_hash

    def _default_hash(self, key):
        # Default hash function using both division and multiplication methods.
        hash1 = key % self.capacity  # Division method
        hash2 = int(self.capacity * ((key * self.A) % 1))  # Multiplication method
        return (hash1 + hash2) % self.capacity

    def _resize(self, new_capacity):
        # Resize the hash table (grow or shrink) and rehash all elements.
        old_table = self.table
        self.capacity = new_capacity
        self.table = [None] * new_capacity
        self.size = 0

        for head in old_table:
            curr = head
            while curr:
                self.insert(curr.key, curr.value)
                curr = curr.next

    def insert(self, key, value):
        if key is None:
            raise ValueError("Key cannot be None")
        
        index = self.hash_function(key)
        new_node = Node(key, value)
        
        if not self.table[index]:
            # If no collision, insert directly.
            self.table[index] = new_node
        else:
            # Collision resolution using chaining (doubly linked list).
            curr = self.table[index]
            while curr:
                if curr.key == key:
                    curr.value = value  # Update existing key
                    return
                if not curr.next:
                    break
                curr = curr.next
            curr.next = new_node
            new_node.prev = curr
        
        self.size += 1
        # Grow the table if it reaches capacity.
        if self.size >= self.capacity:
            self._resize(self.capacity * 2)

    def remove(self, key):
        # Remove a key-value pair from the hash table.
        index = self.hash_function(key)
        curr = self.table[index]

        while curr:
            if curr.key == key:
                # Adjust pointers to remove the node from the doubly linked list.
                if curr.prev:
                    curr.prev.next = curr.next
                if curr.next:
                    curr.next.prev = curr.prev
                if curr == self.table[index]:
                    self.table[index] = curr.next
                self.size -= 1
                # Shrink the table if it becomes 1/4 full (but not below minimum size).
                if self.size <= self.capacity // 4 and self.capacity > 8:
                    self._resize(self.capacity // 2)
                return True
            curr = curr.next
        return False  # Key not found

    def get(self, key):
        # Retrieve a value by its key.
        index = self.hash_function(key)
        curr = self.table[index]
        while curr:
            if curr.key == key:
                return curr.value
            curr = curr.next
        return -1  # Key not found

    def display(self):
        # Print the entire hash table structure.
        for i, head in enumerate(self.table):
            print(f"Bucket {i}: ", end="")
            curr = head
            while curr:
                print(f"({curr.key}, {curr.value}) <-> ", end="")
                curr = curr.next
            print("NULL")


if __name__ == "__main__":
    # Example usage of the HashTable class.
    ht = HashTable()
    ht.insert(1, 10)
    ht.insert(2, 20)
    ht.insert(3, 30)
    ht.insert(4, 40)
    ht.insert(5, 50)
    ht.insert(6, 60)
    ht.insert(7, 70)
    ht.insert(8, 80)
    ht.insert(9, 90)  # Triggers resize
    ht.display()

    print("Value for key 3:", ht.get(3))

    ht.remove(3)
    ht.display()


Bucket 0: (3, 30) <-> NULL
Bucket 1: (6, 60) <-> (9, 90) <-> NULL
Bucket 2: NULL
Bucket 3: NULL
Bucket 4: NULL
Bucket 5: (2, 20) <-> NULL
Bucket 6: (5, 50) <-> NULL
Bucket 7: (8, 80) <-> NULL
Bucket 8: NULL
Bucket 9: NULL
Bucket 10: (1, 10) <-> NULL
Bucket 11: (4, 40) <-> NULL
Bucket 12: (7, 70) <-> NULL
Bucket 13: NULL
Bucket 14: NULL
Bucket 15: NULL
Value for key 3: 30
Bucket 0: NULL
Bucket 1: (6, 60) <-> (9, 90) <-> NULL
Bucket 2: NULL
Bucket 3: NULL
Bucket 4: NULL
Bucket 5: (2, 20) <-> NULL
Bucket 6: (5, 50) <-> NULL
Bucket 7: (8, 80) <-> NULL
Bucket 8: NULL
Bucket 9: NULL
Bucket 10: (1, 10) <-> NULL
Bucket 11: (4, 40) <-> NULL
Bucket 12: (7, 70) <-> NULL
Bucket 13: NULL
Bucket 14: NULL
Bucket 15: NULL


In [9]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

In [11]:

class HashTable:
    def __init__(self, initial_capacity=8, hash_function=None):
        self.capacity = initial_capacity
        self.size = 0
        self.min_capacity = initial_capacity
        self.table = [None] * self.capacity
        self.A = 0.6180339887  # Golden ratio constant
        self.hash_function = hash_function if hash_function else self._default_hash

    def _default_hash(self, key):
        # Multiplication method with golden ratio
        fractional = (key * self.A) % 1
        return int(self.capacity * fractional)

    def _resize(self, new_capacity):
        old_table = self.table
        old_capacity = self.capacity
        self.capacity = new_capacity
        self.table = [None] * new_capacity
        self.size = 0  # Will rebuild size during reinsertion

        for bucket in old_table:
            current = bucket
            while current:
                next_node = current.next  # Preserve next pointer
                self._reinsert_node(current)
                current = next_node

    def _reinsert_node(self, node):
        index = self.hash_function(node.key)
        node.prev = None
        node.next = self.table[index]
        
        if self.table[index]:
            self.table[index].prev = node
        self.table[index] = node
        self.size += 1

    def insert(self, key, value):
        # Check for existing key
        index = self.hash_function(key)
        current = self.table[index]
        
        while current:
            if current.key == key:
                current.value = value
                return
            current = current.next

        # Insert new node at head
        new_node = Node(key, value)
        new_node.next = self.table[index]
        if self.table[index]:
            self.table[index].prev = new_node
        self.table[index] = new_node
        self.size += 1

        # Check if needs to grow
        if self.size >= self.capacity:
            self._resize(self.capacity * 2)

    def remove(self, key):
        index = self.hash_function(key)
        current = self.table[index]

        while current:
            if current.key == key:
                # Remove node from linked list
                if current.prev:
                    current.prev.next = current.next
                else:  # Node is head of list
                    self.table[index] = current.next

                if current.next:
                    current.next.prev = current.prev

                self.size -= 1

                # Check if needs to shrink
                if self.size <= self.capacity // 4 and self.capacity > self.min_capacity:
                    self._resize(max(self.min_capacity, self.capacity // 2))
                return True
            current = current.next
        return False

    def get(self, key):
        index = self.hash_function(key)
        current = self.table[index]
        
        while current:
            if current.key == key:
                return current.value
            current = current.next
        return -1

    def load_factor(self):
        return self.size / self.capacity

    def __str__(self):
        output = []
        for i, bucket in enumerate(self.table):
            entries = []
            current = bucket
            while current:
                entries.append(f"({current.key}:{current.value})")
                current = current.next
            output.append(f"Bucket {i}: {' -> '.join(entries)}" if entries else f"Bucket {i}: Empty")
        return "\n".join(output)


In [15]:
if __name__ == "__main__":
    ht = HashTable()
    
    # Insert test data
    for i in range(15):
        ht.insert(i, i*10)
    
    print("Hash table after insertions:")
    print(ht)
    print(f"Load factor: {ht.load_factor():.2f}")
    
    # Remove test
    for i in range(10):
        ht.remove(i)
    
    print("\nHash table after removals:")
    print(ht)
    print(f"Load factor: {ht.load_factor():.2f}")

    # Custom hash function example
    def custom_hash(key, capacity):
        return (key * 2654435761) % capacity# Fibonacci hashing

    custom_ht = HashTable(hash_function=lambda k: custom_hash(k, custom_ht.capacity))
    custom_ht.insert(42, 100)
    print("\nCustom hash table:", custom_ht.get(42))


Hash table after insertions:
Bucket 0: (13:130) -> (0:0)
Bucket 1: (5:50)
Bucket 2: (10:100)
Bucket 3: (2:20)
Bucket 4: Empty
Bucket 5: (7:70)
Bucket 6: (12:120)
Bucket 7: (4:40)
Bucket 8: (9:90)
Bucket 9: (1:10)
Bucket 10: (14:140)
Bucket 11: (6:60)
Bucket 12: (11:110)
Bucket 13: (3:30)
Bucket 14: Empty
Bucket 15: (8:80)
Load factor: 0.94

Hash table after removals:
Bucket 0: (13:130)
Bucket 1: Empty
Bucket 2: (10:100)
Bucket 3: Empty
Bucket 4: Empty
Bucket 5: Empty
Bucket 6: (12:120)
Bucket 7: Empty
Bucket 8: Empty
Bucket 9: Empty
Bucket 10: (14:140)
Bucket 11: Empty
Bucket 12: (11:110)
Bucket 13: Empty
Bucket 14: Empty
Bucket 15: Empty
Load factor: 0.31


NameError: name 'capacity' is not defined